In [186]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.ops import linemerge, unary_union
from shapely.geometry import LineString, MultiLineString




In [187]:
base_dir  = 'D:/Desktop/MSc Thesis - Copy/'
dir_osm   = 'D:/Desktop/MSc Thesis - Copy/Data/GIS data/south-korea-osm.shp/'
dir_urban = 'D:/Desktop/MSc Thesis - Copy/shapefiles/crop/Urban cropped/'

crop_neig_offset = gpd.read_file(base_dir + 'shapefiles/crop/crop_neighborhood_offset.shp')
railways = gpd.read_file(dir_osm + 'gis_osm_railways_free_1.shp')
railways.to_crs('EPSG:5174', inplace=True)

print(crop_neig_offset.crs)
print(railways.crs)



EPSG:5174
EPSG:5174


In [188]:
railways = railways[railways.intersects(crop_neig_offset.unary_union)]
railways = railways[~railways['name'].isna()]

C:\Users\Michele\AppData\Local\Temp\ipykernel_21068\417759647.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  railways = railways[railways.intersects(crop_neig_offset.unary_union)]


In [189]:
# dissolved = railways.dissolve(by='name', as_index=False)
# cleaned_lines = []


# for idx, row in dissolved.iterrows():
#     name = row['name']
#     geom = row.geometry
#     fclass = row['fclass']

#     # Convert MultiLineString to list of LineStrings
#     if isinstance(geom, LineString):
#         lines = [geom]
#     elif isinstance(geom, MultiLineString):
#         lines = list(geom.geoms)
#     else:
#         continue  # skip unexpected types

#     # Buffer each line slightly (adjust distance as needed)
#     buffers = [line.buffer(20, cap_style='round') for line in lines]  # 80 meters buffer for railways

#     # Keep track of lines to retain
#     to_keep = []

#     for i, line in enumerate(lines):
#         # Check if this line is fully within any other buffer
#         inside_any = False
#         for j, buf in enumerate(buffers):
#             if i == j:
#                 continue
#             if line.within(buf):
#                 inside_any = True
#                 break
#         if not inside_any:
#             to_keep.append(line)

#     # Save the remaining lines as new geometry
#     if len(to_keep) == 1:
#         final_geom = to_keep[0]
#     else:
#         final_geom = MultiLineString(to_keep)

#     # Append to output
#     cleaned_lines.append({
#         'name': name,
#         'geometry': final_geom
#     })

# # Create a new GeoDataFrame
# cleaned_gdf = gpd.GeoDataFrame(cleaned_lines, crs=dissolved.crs)

In [190]:
# united = unary_union(railways.geometry)
# merged = linemerge(united)
# if isinstance(merged, LineString):
#     merged_railways = gpd.GeoDataFrame(geometry=[merged], crs=railways.crs)
# else:  # MultiLineString
#     merged_railways = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=railways.crs)


In [191]:
merged_lines = []

for name, group in railways.groupby('name'):
    # Combine segments into one geometry
    united = unary_union(group.geometry)

    # Ensure input to linemerge is a list or MultiLineString
    if isinstance(united, LineString):
        merged = united  # No need to merge; it's already a single line
    else:
        merged = linemerge(united)

    # Handle both LineString and MultiLineString
    if isinstance(merged, LineString):
        merged_lines.append({'name': name, 'geometry': merged})
    elif isinstance(merged, MultiLineString):
        for part in merged.geoms:
            merged_lines.append({'name': name, 'geometry': part})

merged_railways = gpd.GeoDataFrame(merged_lines, crs=railways.crs)

In [192]:
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString

buffer_distance = 300  # meters

cleaned_lines = []

for name, group in merged_railways.groupby('name'):
    lines = list(group.geometry)
    buffers = [line.buffer(buffer_distance, cap_style=1) for line in lines]

    containment_counts = []

    for i, line in enumerate(lines):
        count = 0
        for j, buf in enumerate(buffers):
            if i != j and line.within(buf):
                count += 1
        containment_counts.append((i, count))

    # Sort lines by how "deeply contained" they are
    containment_counts.sort(key=lambda x: x[1])  # keep least contained lines

    kept = set()
    for i, _ in containment_counts:
        # If not already marked for keeping and doesn't fall within another kept line's buffer
        if all(not lines[i].within(buffers[j]) for j in kept):
            kept.add(i)

    # Ensure at least one line remains
    if not kept and lines:
        kept.add(containment_counts[0][0])  # keep least contained

    final_geoms = [lines[i] for i in kept]
    if len(final_geoms) == 1:
        final_geom = final_geoms[0]
    else:
        final_geom = MultiLineString(final_geoms)

    cleaned_lines.append({
        'name': name,
        'geometry': final_geom
    })

cleaned_gdf = gpd.GeoDataFrame(cleaned_lines, crs=merged_railways.crs)

# buffer_distance = 40  # in meters, adjust based on CRS (20m for EPSG:5174 is fine)

# cleaned_lines = []

# for name, group in merged_railways.groupby('name'):
#     lines = list(group.geometry)

#     # Generate buffer for each line (circular buffer)
#     buffers = [line.buffer(buffer_distance, cap_style=1) for line in lines]

#     to_keep = []
#     for i, line in enumerate(lines):
#         inside_any = False
#         for j, buf in enumerate(buffers):
#             if i == j:
#                 continue
#             if line.within(buf):
#                 inside_any = True
#                 break
#         if not inside_any:
#             to_keep.append(line)

#     # Combine retained lines into a MultiLineString (or LineString)
#     if len(to_keep) == 1:
#         final_geom = to_keep[0]
#     elif len(to_keep) > 1:
#         final_geom = MultiLineString(to_keep)
#     else:
#         continue  # Skip if nothing remains

#     cleaned_lines.append({
#         'name': name,
#         'geometry': final_geom
#     })

# cleaned_gdf = gpd.GeoDataFrame(cleaned_lines, crs=merged_railways.crs)

In [193]:
drop_names = [
    '경부제1본선', '경부제3본선', '경부선', '용산역 인상선',
    '경부제3본선', '경인선', '경부제2선', '경인제2본',
    '경부선', '경인제2본선', '경부선', '구로기지선',
    '사당역 인상선', '남태령역 인상선'
]
cleaned_gdf = cleaned_gdf[~cleaned_gdf['name'].isin(drop_names)]


In [194]:


cleaned_gdf.to_file(base_dir + 'Data/GIS data/Urban cropped/tests/railways_cropped10.shp', driver='ESRI Shapefile')